# cards.png ダイヤ検出チューニング（OpenCV / Python）

このノートブックは、AIVisionBox の C++ 実装（HSV→赤抽出2レンジ→morphology→輪郭→面積フィルタ）を **Pythonで高速にチューニング**するためのものです。

- 画像は OpenCV 公式サンプル `cards.png` を取得（初回のみDL）
- `debug_mask.png` / `debug_contours.png` 相当を **Notebook上で可視化**
- 閾値（HSV / morphology / area）を変えて、結果を見ながら最適化

> 目的：Pythonで探索 → 良さそうなパラメータを C++ にフィードバック


##準備

###１．ライブラリインストール

In [ ]:
#!pip -q install opencv-python numpy matplotlib
from __future__ import annotations

import os
from pathlib import Path
import urllib.request

import cv2
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

##準備

###２．画像取得

画像取得用関数

In [ ]:
CARDS_URL = "https://raw.githubusercontent.com/opencv/opencv/4.x/samples/data/cards.png"

# practice は単体で動くのが目的なので、固定パスでOK
cards_path = Path("assets/opencv/cards.png")
cards_path.parent.mkdir(parents=True, exist_ok=True)

# 画像が無ければダウンロード
if not cards_path.exists():
    urllib.request.urlretrieve(CARDS_URL, str(cards_path))

print(f"exists({cards_path})={int(cards_path.exists())}")

img_bgr = cv2.imread(str(cards_path), cv2.IMREAD_COLOR)
assert img_bgr is not None and img_bgr.size > 0, "cv2.imread failed"

img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)



##準備

##3.ROI（関心領域）

v0.1.x では **画像全体**でダイヤ総数を数える前提ですが、
チューニングでは ROI を使うと安定しやすいです（まずは全体→必要なら部分）。

- ROI = `(x, y, w, h)`
- `w=0 or h=0` のとき全体扱い


In [ ]:
def clamp_roi(roi: tuple[int,int,int,int], width: int, height: int) -> tuple[int,int,int,int]:
    x, y, w, h = roi
    x = max(0, int(x)); y = max(0, int(y))
    w = max(0, int(w)); h = max(0, int(h))
    if w == 0 or h == 0:
        return (0, 0, width, height)
    if x >= width or y >= height:
        return (0, 0, width, height)
    x2 = min(width, x + w)
    y2 = min(height, y + h)
    if x2 <= x or y2 <= y:
        return (0, 0, width, height)
    return (x, y, x2 - x, y2 - y)

ROI関数の動作検証

In [ ]:
# まずは全体(画像サイズを使用する)
roi = (0, 0, img_bgr.shape[1], img_bgr.shape[0])

x, y, w, h = clamp_roi(roi, img_bgr.shape[1], img_bgr.shape[0])
roi_rgb = img_rgb[y:y+h, x:x+w]

#グラフに画像出力
plt.figure(figsize=(10,6))
plt.imshow(roi_rgb)
plt.title(f"ROI: x={x}, y={y}, w={w}, h={h}")
plt.axis("off")
plt.show()

## コア処理（C++ CountBgr24 相当）

C++ 実装と同じ流れ：

1. ROI切り出し
2. HSV 変換
3. 赤抽出（2レンジ）
4. morphology（OPEN 1回 + CLOSE 強め）
5. findContours
6. 面積フィルタ（minArea / maxArea）

Notebookでは以下を **可視化**します：
- HSV 赤抽出マスク
- morphology 後マスク
- 輪郭描画結果（フィルタ通過のみ）
- 面積分布（上位表示）


In [ ]:
def run_pipeline(
    img_bgr: np.ndarray,
    roi: tuple[int,int,int,int],
    h1=(0, 10),
    h2=(170, 180),
    s_min=80,
    v_min=80,
    open_ksize=3,
    open_iter=1,
    close_ksize=5,
    close_iter=2,
    min_area=1200,
    max_area=20000,
):
    #対象範囲の限定(ROI切り出し)
    H, W = img_bgr.shape[:2]
    x, y, w, h = clamp_roi(roi, W, H)
    roi_mat = img_bgr[y:y+h, x:x+w].copy()

    #BGR→HSVに変換
    hsv = cv2.cvtColor(roi_mat, cv2.COLOR_BGR2HSV)

    #赤の範囲(2レンジ作成する)
    #h1=(0,10), h2=(170,180)：典型的な赤
    #s_min：彩度（薄い赤/ピンクを除外する）
    #v_min：明度（暗い赤/影を除外する）
    lower1 = (h1[0], s_min, v_min)
    upper1 = (h1[1], 255, 255)
    lower2 = (h2[0], s_min, v_min)
    upper2 = (h2[1], 255, 255)

    #赤マスクを作る（2レンジ抽出 → 合成）
    #inRange は「条件内なら255、外なら0」の 白黒マスクを作ります。
    #mask_raw が「赤っぽい部分」だけ白になった画像です。
    mask1 = cv2.inRange(hsv, lower1, upper1)
    mask2 = cv2.inRange(hsv, lower2, upper2)
    mask_raw = cv2.bitwise_or(mask1, mask2)

    #rawマスクの白画素数をログ（量の指標）
    #抽出できた“赤画素の量”の目安。
    #s_min/v_min を上げると減り、下げると増えます。
    nz_raw = int(cv2.countNonZero(mask_raw))

    #OPEN（小さいノイズ除去：削る方向）
    #OPEN = erode→dilate
    #小さい点ノイズを消したり、細いヒゲを削って「綺麗な塊」にします。
    #open_ksize（カーネルサイズ）大きいほど強く削る
    #open_iter（回数）多いほど強く削る
    mask = mask_raw
    if open_ksize > 0 and open_iter > 0:
        k_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (open_ksize, open_ksize))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k_open, iterations=open_iter)
    #CLOSE（分断された領域を結合：くっつける方向）
    #CLOSE = dilate→erode
    #割れた赤領域を 繋げて1つの塊に寄せる 効果があります。
    #cards.png は赤が割れやすいので、ここが効きどころ。
    #close_ksize 大きいほど“つなげ力”が増える
    #close_iter 多いほど“つなげ力”が増える

    if close_ksize > 0 and close_iter > 0:
        k_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (close_ksize, close_ksize))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k_close, iterations=close_iter)
    #morphedマスクの白画素数をログ
    #nz_raw と比べて「ノイズ除去/結合でどう変わったか」を見る指標。
    #OPENで減り、CLOSEで増えることが多いです。
    nz = int(cv2.countNonZero(mask))

    #輪郭抽出（白い塊を個体として拾う）
    #白い塊を輪郭（contour）として取得します。
    #RETR_EXTERNAL なので「外側の輪郭だけ」拾います（内側穴は無視）。
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    areas = [float(cv2.contourArea(c)) for c in contours]

    #面積フィルタで“ダイヤらしいサイズ”だけ残す
    #min_area 未満 → 小ノイズ扱いで捨てる
    #max_area 超え → 大きな赤塊（重なり・カード端など）として捨てる
    #いま area_max が 257 みたいな状況なら、min_area=1200 は絶対に残らないので count=0 になります。
    #つまり この値は“画像と処理結果の面積スケール”に合わせる必要があるです。
    kept = []
    for c, a in zip(contours, areas):
        if a < min_area or a > max_area:
            continue
        kept.append(c)

    vis = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    cv2.drawContours(vis, kept, -1, (0, 255, 0), 2)

    info = {
        "roi": (x, y, w, h),
        "mask_nonzero_raw": nz_raw,
        "mask_nonzero": nz,
        "contours_total": len(contours),
        "count_kept": len(kept),
        "area_min": min(areas) if areas else None,
        "area_max": max(areas) if areas else None,
    }
    # ★ kept を追加して返す
    return roi_mat, mask_raw, mask, vis, areas, info, kept


面積分布（ヒストグラム）を見るコード

「小さいダイヤ群」と「大きいダイヤ群」が分離しているなら、min_area が効きます。

logy=True にすると、小さい面積の山と大きい面積の山が見やすいです。

パーセンタイルを見ると「min_area候補」が決めやすいです。

In [ ]:
def plot_area_hist(areas, bins=60, logy=True):
    if not areas:
        print("areas is empty")
        return
    a = np.array(areas, dtype=float)

    plt.figure(figsize=(8,4))
    plt.hist(a, bins=bins)
    plt.title("Contour area histogram")
    plt.xlabel("area")
    plt.ylabel("count")
    if logy:
        plt.yscale("log")
    plt.grid(True, which="both", axis="y")
    plt.show()

kept に番号付きで描画（答え合わせが一瞬）

「44個が本当に欲しい対象か」を一発で確認できます。

In [ ]:
def draw_kept_with_index(roi_mat, kept, color=(0, 255, 0)):
    """
    kept（輪郭リスト）を番号付きで描画する。
    """
    vis = roi_mat.copy()

    for i, c in enumerate(kept):
        # 輪郭描画
        cv2.drawContours(vis, [c], -1, color, 2)

        # 重心計算（番号表示位置）
        M = cv2.moments(c)
        if M["m00"] > 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
        else:
            x, y, w, h = cv2.boundingRect(c)
            cx = x + w // 2
            cy = y + h // 2

        cv2.putText(
            vis,
            str(i),
            (cx, cy),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.5,
            (255, 255, 0),
            1,
            cv2.LINE_AA,
        )

    return vis

1：Params（dataclass）＋JSON保存/読込ユーティリティ

In [ ]:
import json
from dataclasses import dataclass, asdict
from pathlib import Path

@dataclass
class Params:
    # ROI: (x, y, w, h)
    roi: tuple[int, int, int, int]

    # HSV threshold for "red" (two ranges)
    h1: tuple[int, int] = (0, 12)
    h2: tuple[int, int] = (168, 179)  # OpenCV HSV H is 0..179

    s_min: int = 60
    v_min: int = 60

    # Morphology
    open_ksize: int = 3
    open_iter: int = 1
    close_ksize: int = 7
    close_iter: int = 2

    # Area filter
    min_area: int = 50
    max_area: int = 5000

def save_params_json(params: Params, path: str | Path = "params_cards.json") -> Path:
    path = Path(path)
    path.write_text(json.dumps(asdict(params), indent=2), encoding="utf-8")
    print(f"[saved] {path.resolve()}")
    return path

def load_params_json(path: str | Path = "params_cards.json") -> Params:
    path = Path(path)
    d = json.loads(path.read_text(encoding="utf-8"))
    # tupleに戻す（JSONはlistになるため）
    d["roi"] = tuple(d["roi"])
    d["h1"] = tuple(d["h1"])
    d["h2"] = tuple(d["h2"])
    p = Params(**d)
    print(f"[loaded] {path.resolve()}")
    return p

def print_params_for_cpp(params: Params) -> None:
    d = asdict(params)
    print("=== FINAL PARAMS (copy to C++) ===")
    print(f"roi: x={d['roi'][0]}, y={d['roi'][1]}, w={d['roi'][2]}, h={d['roi'][3]}")
    print(f"h1: ({d['h1'][0]}..{d['h1'][1]}), h2: ({d['h2'][0]}..{d['h2'][1]})")
    print(f"s_min={d['s_min']} v_min={d['v_min']}")
    print(f"open: k={d['open_ksize']} iter={d['open_iter']}")
    print(f"close: k={d['close_ksize']} iter={d['close_iter']}")
    print(f"area: min={d['min_area']} max={d['max_area']}")


2：可視化ユーティリティ（4枚表示＋番号付き表示）

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

def show_debug_4(roi_mat, mask_raw, mask, vis, info):
    fig, ax = plt.subplots(1, 4, figsize=(18, 5))
    ax[0].imshow(cv2.cvtColor(roi_mat, cv2.COLOR_BGR2RGB)); ax[0].set_title("ROI"); ax[0].axis("off")
    ax[1].imshow(mask_raw, cmap="gray"); ax[1].set_title(f"mask raw (nz={info['mask_nonzero_raw']})"); ax[1].axis("off")
    ax[2].imshow(mask, cmap="gray"); ax[2].set_title(f"mask morphed (nz={info['mask_nonzero']})"); ax[2].axis("off")
    ax[3].imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)); ax[3].set_title(f"kept contours: {info['count_kept']}"); ax[3].axis("off")
    plt.show()

def show_indexed(roi_mat, kept):
    vis_idx = draw_kept_with_index(roi_mat, kept)
    plt.figure(figsize=(7, 7))
    plt.imshow(cv2.cvtColor(vis_idx, cv2.COLOR_BGR2RGB))
    plt.title(f"kept indexed: {len(kept)}")
    plt.axis("off")
    plt.show()

3：run_pipeline の返り値差異に強い「ラッパー」

In [ ]:
def run_pipeline_compat(img_bgr, **params):
    """
    run_pipeline の戻り値が 6個 or 7個（kept付き）どちらでも吸収するラッパー。
    """
    out = run_pipeline(img_bgr, **params)

    if isinstance(out, tuple) and len(out) == 7:
        roi_mat, mask_raw, mask, vis, areas, info, kept = out
    elif isinstance(out, tuple) and len(out) == 6:
        roi_mat, mask_raw, mask, vis, areas, info = out
        # kept が無い場合、ここでは None にする（番号描画はスキップ）
        kept = None
    else:
        raise ValueError(f"Unexpected run_pipeline return: type={type(out)} len={len(out) if isinstance(out, tuple) else 'N/A'}")

    return roi_mat, mask_raw, mask, vis, areas, info, kept


4：1セル完結の本体 run_and_visualize()

In [ ]:
def run_and_visualize(
    img_bgr: np.ndarray,
    params: Params,
    *,
    bins: int = 60,
    logy: bool = True,
    save_params: bool = True,
    params_path: str = "params_cards.json",
):
    # Params -> dict（run_pipeline互換）
    p = asdict(params)

    roi_mat, mask_raw, mask, vis, areas, info, kept = run_pipeline_compat(img_bgr, **p)
    print("[dbg]", info)

    # 4枚表示
    show_debug_4(roi_mat, mask_raw, mask, vis, info)

    # 面積の上位表示
    if areas:
        areas_sorted = sorted(areas, reverse=True)
        print("top areas:", [round(a, 1) for a in areas_sorted[:20]])

        # ヒストグラム + パーセンタイル
        plot_area_hist(areas, bins=bins, logy=logy)
        print("area percentiles:", np.percentile(np.array(areas), [50, 75, 90, 95, 99]))

    # kept番号付き（keptがある場合のみ）
    if kept is not None:
        show_indexed(roi_mat, kept)
    else:
        print("[info] kept is not returned by run_pipeline. (update run_pipeline to return kept)")

    # Params保存（再現性のため）
    if save_params:
        save_params_json(params, params_path)
        print_params_for_cpp(params)

    # 返り値（あとで比較・保存にも使える）
    return {
        "roi_mat": roi_mat,
        "mask_raw": mask_raw,
        "mask": mask,
        "vis": vis,
        "areas": areas,
        "info": info,
        "kept": kept,
        "params": params,
    }


セル5：実行例（これだけで回る）

In [ ]:
# 例：あなたの ROI をそのまま使う
p = Params(
    roi=roi,
    h1=(0, 12),
    h2=(168, 179),
    s_min=60,
    v_min=60,
    open_ksize=3,
    open_iter=1,
    close_ksize=7,
    close_iter=2,
    min_area=50,
    max_area=5000,
)

out = run_and_visualize(img_bgr, p)
